# Building a deep multi layer perceptron for the MNIST dataset with all the bells and whistles.

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
print(mnist)

Datasets(train=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x104009780>, validation=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0xb23868668>, test=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0xb238682e8>)


In [3]:
import tensorflow as tf

In [4]:
n_inputs = 28*28 #MNIST
n_hidden1 = 1000
n_hidden2 = 500
n_hidden3 = 300
n_hidden4 = 200
n_hidden5 = 100
n_outputs = 10

In [5]:
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")
def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2 / np.sqrt(n_inputs + n_neurons)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        W = tf.Variable(init, name="kernel")
        b = tf.Variable(tf.zeros([n_neurons]), name="bias")
        Z = tf.matmul(X, W) + b
        if activation is not None:
            return activation(Z)
        else:
            return Z

In [6]:
with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1",
                          activation=tf.nn.relu)
    hidden2 = tf.layers.dense(hidden1, n_hidden2, name="hidden2",
                          activation=tf.nn.relu)
    hidden3 = tf.layers.dense(hidden2, n_hidden3, name="hidden3",
                          activation=tf.nn.relu)
    hidden4 = tf.layers.dense(hidden3, n_hidden4, name="hidden4",
                          activation=tf.nn.relu)
    hidden5 = tf.layers.dense(hidden4, n_hidden5, name="hidden5",
                          activation=tf.nn.relu)
    logits = tf.layers.dense(hidden5, n_outputs, name="outputs")

Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Colocations handled automatically by placer.


In [7]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,
                                                             logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

In [9]:
learning_rate = 0.01
with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [10]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [11]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [12]:
n_epochs = 50
batch_size = 50

In [13]:
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_val = accuracy.eval(feed_dict={X: mnist.validation.images, y: mnist.validation.labels})
        print(epoch, "Train accuracy:", acc_train, "Val accuracy", acc_val)
    save_path = saver.save(sess, "./my_model_final.ckpt")

0 Train accuracy: 0.98 Val accuracy 0.9216
1 Train accuracy: 0.98 Val accuracy 0.9408
2 Train accuracy: 0.94 Val accuracy 0.9516
3 Train accuracy: 1.0 Val accuracy 0.964
4 Train accuracy: 0.98 Val accuracy 0.9612
5 Train accuracy: 1.0 Val accuracy 0.9676
6 Train accuracy: 1.0 Val accuracy 0.9684
7 Train accuracy: 1.0 Val accuracy 0.973
8 Train accuracy: 0.98 Val accuracy 0.9764
9 Train accuracy: 1.0 Val accuracy 0.9772
10 Train accuracy: 1.0 Val accuracy 0.977
11 Train accuracy: 1.0 Val accuracy 0.9808
12 Train accuracy: 1.0 Val accuracy 0.9782
13 Train accuracy: 1.0 Val accuracy 0.9786
14 Train accuracy: 1.0 Val accuracy 0.9798
15 Train accuracy: 1.0 Val accuracy 0.9798
16 Train accuracy: 1.0 Val accuracy 0.9798
17 Train accuracy: 1.0 Val accuracy 0.9772
18 Train accuracy: 1.0 Val accuracy 0.9792
19 Train accuracy: 1.0 Val accuracy 0.9798
20 Train accuracy: 1.0 Val accuracy 0.9766
21 Train accuracy: 1.0 Val accuracy 0.9804
22 Train accuracy: 1.0 Val accuracy 0.979
23 Train accuracy: 1

In [14]:
def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script src="//cdnjs.cloudflare.com/ajax/libs/polymer/0.3.3/platform.js"></script>
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [23]:
from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

In [24]:
with tf.name_scope("eval") as scope:
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
accuracy_summary = tf.summary.scalar('Accuracy', accuracy)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [25]:
import numpy as np

In [28]:
# start session
saver = tf.train.Saver()

batch_size = 50
n_batches = int(np.ceil(n_inputs / batch_size))

def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)  
    indices = np.random.randint(n_inputs, size=batch_size)  
    X_batch = x[indices] 
    y_batch = y[indices] 
    return X_batch, y_batch
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        if epoch % 5 == 0 and epoch !=0:
            summary_str = accuracy_summary.eval(feed_dict={X: X_batch, y: y_batch})
            step = epoch * n_batches + epoch
            file_writer.add_summary(summary_str, step)
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_val = accuracy.eval(feed_dict={X: mnist.validation.images, y: mnist.validation.labels})
        print(epoch, "Train accuracy:", acc_train, "Val accuracy", acc_val)
    save_path = saver.save(sess, "./my_model_final.ckpt")

0 Train accuracy: 0.96 Val accuracy 0.9266
1 Train accuracy: 1.0 Val accuracy 0.9452
2 Train accuracy: 1.0 Val accuracy 0.957
3 Train accuracy: 1.0 Val accuracy 0.9636
4 Train accuracy: 1.0 Val accuracy 0.967
5 Train accuracy: 1.0 Val accuracy 0.971
6 Train accuracy: 1.0 Val accuracy 0.9744
7 Train accuracy: 1.0 Val accuracy 0.9758
8 Train accuracy: 1.0 Val accuracy 0.9748
9 Train accuracy: 1.0 Val accuracy 0.9772
10 Train accuracy: 1.0 Val accuracy 0.9756
11 Train accuracy: 1.0 Val accuracy 0.9776
12 Train accuracy: 1.0 Val accuracy 0.979
13 Train accuracy: 1.0 Val accuracy 0.9794
14 Train accuracy: 1.0 Val accuracy 0.9784
15 Train accuracy: 1.0 Val accuracy 0.9796
16 Train accuracy: 1.0 Val accuracy 0.9796
17 Train accuracy: 1.0 Val accuracy 0.9806
18 Train accuracy: 1.0 Val accuracy 0.9818
19 Train accuracy: 1.0 Val accuracy 0.9806
20 Train accuracy: 1.0 Val accuracy 0.9812
21 Train accuracy: 1.0 Val accuracy 0.98
22 Train accuracy: 1.0 Val accuracy 0.9804
23 Train accuracy: 1.0 Val

In [29]:
file_writer.close()